In [ ]:
import os
from enum import Enum
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate  # <-- Add PromptTemplate here
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

# .env 파일 로드
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. 감정 상태를 정의하는 Enum 클래스
class Sentiment(Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# 2. EnumOutputParser 초기화
output_parser = EnumOutputParser(enum=Sentiment)

# 3. 모델 초기화
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  
    model="meta-llama/llama-4-scout-17b-16e-instruct",  
    temperature=0.7
)

# 4. 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""영화 리뷰 감정을 세 가지 중 하나로 분류해주세요: 긍정, 부정, 보통.
다른 부연 설명 없이, 다음 응답 중 정확히 하나만 출력하세요.
리뷰: "{review}"
{format_instructions}""",
    input_variables=["review"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# 5. LLMChain 구성 (프롬프트 + 모델)
chain = prompt | llm | output_parser

# 6. 테스트 리뷰 목록
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
]

# 7. 리뷰에 대해 감정 분석 실행 및 결과 출력
print("--- 영화 리뷰 감정 분석기 ---")
for review in test_reviews:
    print(f"\n[리뷰]: {review}")
    try:
        # 체인 실행
        sentiment = chain.invoke({"review": review})
        print(f"[감정 분석 결과]: {sentiment.value}")
    except Exception as e:
        print(f"[분석 실패]: {e}")

--- 영화 리뷰 감정 분석기 ---

[리뷰]: 이 영화 정말 재미없어요. 시간 낭비였습니다.
[감정 분석 결과]: 부정

[리뷰]: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
[감정 분석 결과]: 긍정

[리뷰]: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
[감정 분석 결과]: 보통
